### Optimizing polynomials with strange ways

Author: Raido Everest

In [ ]:
# Scipy minimize appears to work locally - depending on the starting point, we may arrive
# at a different end position. This might bias end results toward methods that initialize
# in many different places, but I highly doubt it makes much of a difference here.

import random
import numpy as np
from scipy.optimize import minimize  # checking goodness of result compared to scipy

In [ ]:
# Function to generate functions to test on
def quartic():
    # default ranges for variables too
    a, b = random.random() * 4.9 + 0.1, random.random() * 10 - 5
    c, d = random.random() * 40 - 20, random.random() * 500 - 250
    
    return lambda x: a * x**4 + b * x**3 + c * x**2 + d * x

# Making an nd-quartic function to test higher dimensionalities.
# For the sake of generality, this should be used for the single variable case as well.
# n - how many inputs the function takes
# outputs the sum of n random quartic functions
def quartic_n(n):
    fs = [quartic() for _ in range(n)]
    return lambda answers: sum(map(lambda pair: pair[0](pair[1]), zip(fs, answers)))

# Example scipy optimization of a 4d quartic function:
# minimize(quartic_n(4), (0,0,0,0))

### Differential Evolution

In [ ]:
# The main function that returns the best set of inputs it finds.
# f - the function.
# n - input dimensionality
# k - population size
# scaling - the scaling parameter when creating a new input set
# loops   - how many loops it will do before giving up finding a better solution
# Outputs the minimum value of the function that it found and the necessary input for it
def diff_evo(f, n, k = 100, scaling = 0.5, loops=25):
    # Create initial input set
    pop       = create_population(n, k)
    fitnesses = calculate_fitness(f, pop)
    best, bestval = getbest(pop, fitnesses)   # pair of best input and its index
    
    loops_since_improvement = 0  # Keep it going until it's not working anymore.
    while loops_since_improvement < loops:
        loops_since_improvement += 1
        
        # Create next population by mutating previous one
        newpop       = create_next_population(pop, scaling)
        newfitnesses = calculate_fitness(f, newpop)
        nextbest, nextbestval = getbest(newpop, newfitnesses)
        
        # Keep track of what the best outcome is
        if nextbestval < bestval:
            best, bestval = nextbest, nextbestval
            loops_since_improvement = 0
        
        # Always choose the better one of the two choices to represent the 'next generation'
        for i in range(k):
            if newfitnesses[i] < fitnesses[i]:  # if something must be changed
                pop[i], fitnesses[i] = newpop[i], newfitnesses[i]
    
    # Return the best value and its inputs
    return bestval, best

# Creates a population not knowing any previous information
# n - dimensionality of output
# k - population size
def create_population(n, k):
    return [[random.random() * 200 - 100 for _ in range(n)] for _ in range(k)]

# Creates the next generation of input sets
# pop - the old population
# scaling - the scaling parameter
def create_next_population(pop, scaling = 0.5):
    dim = len(pop[0])
    n = len(pop)
    
    newpop = [None] * n
    for i in range(n):
        a, b = random.randint(0, n-1), random.randint(0, n-1)  # Indices of two random elements
        diff = [(pop[a][d] - pop[b][d]) for d in range(dim)]   # Difference of two random input vectors
        newpop[i] = [pop[i][d] + diff[d] * scaling for d in range(dim)]  # Mutated input has been created
    
    return newpop

# Just makes a list of evaluation results
def calculate_fitness(f, pop):
    return [f(inputs) for inputs in pop]

# Given a population and fitnesses, returns the best element and its fitness.
def getbest(pop, fitnesses):
    best, bestfitness = pop[0], fitnesses[0]
    for i in range(1, len(fitnesses)):
        if fitnesses[i] < bestfitness:
            best, bestfitness = pop[i], fitnesses[i]
    return best, bestfitness

### Testing Goodness of Differential Evolution

I expect scipy and diffevo to be pretty close overall.

In [ ]:
# Generate a bunch of functions. Use scipy and diff_evo to find optimal solutions.
# Arbitrarily choose amount of functions for each dimensionality.
functions = 10

print('Measuring difference of scipy and diff_evo - the higher, the better, because that means DE works fine...')
for dimensions in range(1,5):
    print(f'Testing out {dimensions} dimensions...')
    for _ in range(functions):
        function = quartic_n(dimensions)

        sp_ans = minimize(function, [0] * dimensions)
        sp_bestval, sp_bestinput = sp_ans.fun, sp_ans.x

        de_bestval, de_bestinput = diff_evo(function, dimensions)

        print(f'Difference between scipy and diffevo: {sp_bestval - de_bestval}')
    print()